# This notebook show my issue with the calculation time of the enrichment function

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
import greatpy as great
import pandas as pd

In [3]:
test = pd.read_csv("../data/human/test_genomic_region.bed",sep="\t",comment="#",
                names=["Chr", "Chr_Start", "Chr_End"],dtype={"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64"})

df_10_row = test[:30]

In [4]:
%lprun -f great.tl.enrichment great.tl.enrichment(df_10_row,"../data/human/regulatory_domain.bed","../data/human/chr_size.bed","../data/human/ontologies.csv",True,True,0.05,(0,0))

Timer unit: 1e-06 s

Total time: 9.00199 s
File: /home/tom/.local/lib/python3.8/site-packages/greatpy/tl/basic.py
Function: enrichment at line 312

Line #      Hits         Time  Per Hit   % Time  Line Contents
   312                                           def enrichment(test:str or pd.DataFrame,regdom_file,chr_size_file,annotation,binom=True,hypergeom=True,alpha=0.05,correction=("fdr",0.05),sort_by=None): 
   313                                               # Data import 
   314         1         42.0     42.0      0.0      if not binom and not hypergeom : 
   315                                                   return False
   316                                               
   317         2      83626.0  41813.0      0.9      regdom = pd.read_csv(regdom_file,sep="\t",comment="#",
   318         1          5.0      5.0      0.0                      names=["Chr", "Chr_Start", "Chr_End","Name","tss","Strand"],dtype={"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64","Name":"

Here we can see that the calculation time of the enrichment function is very high due to : 

line 341 (50 %) in the https://github.com/theislab/greatpy/blob/correction/greatpy/tl/basic.py with k_binom = number_of_hit(test,curr_regdom) => The number_of_hit function is used to calculate the number of genomic region hit with each GO term 

line 351 (30 %) https://github.com/theislab/greatpy/blob/correction/greatpy/tl/basic.py with dict.update with the values of binom p-value and hypergeom p-value 

In [14]:
great.tl.enrichment(df_10_row,"../data/human/regulatory_domain.bed","../data/human/chr_size.bed","../data/human/ontologies.csv",True,True,0.05,(0,0)).replace(0,np.nan).sort_values("hypergeom_p_value")

,go_term,binom_p_value,hypergeom_p_value
GO:0045887,positive regulation of synaptic growth at neur...,3.08912e-15,6.45772e-04
GO:0036250,peroxisome transport along microtubule,2.90091e-12,6.45772e-04
GO:0044721,"protein import into peroxisome matrix, substra...",2.90091e-12,6.45772e-04
GO:0016561,"protein import into peroxisome matrix, translo...",3.78579e-12,1.29116e-03
GO:1901094,negative regulation of protein homotetrameriza...,1.52937e-11,1.29116e-03
...,...,...,...
GO:0005576,extracellular region,4.35887e-01,9.99837e-01
GO:0005886,plasma membrane,8.79471e-01,9.99924e-01
GO:0005654,nucleoplasm,8.50718e-01,1.00000e+00
GO:0005515,protein binding,9.89369e-01,NaN
